In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
from sklearn.decomposition import PCA
import pickle
pd.set_option("display.max_columns", None)

In [2]:
store = pd.HDFStore('predict_data.h5')
df = store['predict_data']


In [3]:
filename = 'xgb_model_default.sav'
default_model = pickle.load(open(filename, 'rb'))

In [6]:
filename = 'xgb_model_prepay.sav'
prepay_model = pickle.load(open(filename, 'rb'))

In [7]:
appended_data = []
for chunk in np.array_split(df, 10):
    df_rep = chunk.iloc[np.repeat(np.arange(len(chunk)), chunk["term"])]
    
    df_mob = pd.DataFrame(df_rep, columns= ['unique_id'])
    df_mob['mob'] = df_mob.groupby("unique_id").cumcount()+1
    
    df_combine = pd.concat([df_rep, df_mob], axis=1)
    X = df_combine.drop(['unique_id'], axis=1)
    
    encoder_1 = LabelEncoder()
    # Fit the encoder on the data "platform_id"
    encoder_1.fit(X["platform_id"])
    X_platform_id_encoded = encoder_1.transform(X["platform_id"])
    X["platform_id"] = X_platform_id_encoded

    # Fit the encoder on the data "grade"
    encoder_2 = LabelEncoder()
    encoder_2.fit(X["grade"])
    X_grade_encoded = encoder_2.transform(X["grade"])
    X["grade"] = X_grade_encoded

    # Fit the encoder on the data "home_ownership"
    encoder_3 = LabelEncoder()
    encoder_3.fit(X["home_ownership"])
    X_home_ownership_encoded = encoder_3.transform(X["home_ownership"])
    X["home_ownership"] = X_home_ownership_encoded
    
    X_mob = X['mob'].reset_index(drop=True)
    X = X.drop('mob', axis=1)
    X_year = X['Orig_year'].reset_index(drop=True)
    X = X.drop('Orig_year', axis=1)
    X_month = X['Orig_month'].reset_index(drop=True)
    X = X.drop('Orig_month', axis=1)    
    
    
    sc = StandardScaler()
    X_scaled = sc.fit_transform(X.fillna(0))

    # Apply PCA
    pca = PCA(n_components= 11)
    X_pca = pd.DataFrame(pca.fit_transform(X_scaled))
    
    X_pca = pd.concat([X_pca, X_mob,X_year,X_month], axis=1)
    # predict probabilities
    default_probs = default_model.predict_proba(X_pca)
    # keep probabilities for the positive outcome only
    default_probs = default_probs[:, 1]
    df_combine['CO_Probability'] = default_probs
    
    prepay_probs = prepay_model.predict_proba(X_pca)
    # keep probabilities for the positive outcome only
    prepay_probs = prepay_probs[:, 1]
    df_combine['PrePmt_Probability'] = prepay_probs
    
    appended_data.append(df_combine[['unique_id','mob','CO_Probability','PrePmt_Probability']].reset_index(drop=True))
    print("ok")
appended_data = pd.concat(appended_data)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

ok


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

ok


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

...

ok


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

ok


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

ok


In [8]:
prediction = appended_data.loc[:,~appended_data.columns.duplicated()]

In [13]:
prediction[prediction['unique_id']=='A-675206-232']

,unique_id,mob,CO_Probability,PrePmt_Probability
9876,A-675206-232,1,0.000034,0.009255
9877,A-675206-232,2,0.000042,0.009255
9878,A-675206-232,3,0.000154,0.009255
9879,A-675206-232,4,0.000169,0.010221
9880,A-675206-232,5,0.001628,0.010221
9881,A-675206-232,6,0.002988,0.011687
9882,A-675206-232,7,0.003799,0.014406
9883,A-675206-232,8,0.004707,0.014406
9884,A-675206-232,9,0.005939,0.014406
9885,A-675206-232,10,0.007917,0.017456


In [9]:
prediction.to_csv('prediction.csv', index = False)